```
select distinct count(*) over (partition by color), color from production.product

select 
	distinct sd.productid, 
	p."name",
	count(*) over (partition by sd.productid) as n_orders,
	count(*) over (),
	100.0 * count(*) over (partition by sd.productid) / count(*) over () as shares  
from salesorderdetail as sd
join product as p on p.productid =sd.productid 
order by 5 desc
```
---

```
select distinct pc."name", date_part('y', sh.duedate),
date_part('mon', sh.duedate), 
sum(sd.orderqty*sd.unitprice) over yr,
sum(sd.orderqty*sd.unitprice) over mon
from
salesorderheader as sh
join salesorderdetail as sd using (salesorderid)
join product as p using(productid)
join productsubcategory as psc using(productsubcategoryid)
join productcategory as pc using(productcategoryid)
window yr as (partition by date_part('y', sh.duedate), p.productsubcategoryid), 
mon as (partition by date_part('y', sh.duedate), date_part('mon', sh.duedate), p.productsubcategoryid)
order by 2 desc, 3 desc, 1
```

--- 
window function can not nested each other. but can use subquery
```
select productid, rank() over (order by order_number desc) 
	from (select distinct productid, count(*) over (partition by productid) as order_number from salesorderdetail as sd) as data
    
select productid, dense_rank() over (order by order_number desc) 
	from (select distinct productid, count(*) over (partition by productid) as order_number from salesorderdetail as sd) as data
    
```

row_number is unique
```
select productid, row_number() over (order by order_number desc) 
	from (select distinct productid, count(*) over (partition by productid) as order_number from salesorderdetail as sd) as data
```

Find products, which have 5 greatest numbers of finished manufacturing orders in every
quarter. Exclude orders with scrapped items

```
select * from (
select
quarter,
productid,
number_of_orders,
dense_rank()
OVER(partition by quarter order by number_of_orders desc) as r
from
(select count(*) as number_of_orders,
''|| date_part('y', w.enddate) || '_' || date_part('quarter', w.enddate) as
quarter, w.productid
from production.workorder as w
where scrappedqty = 0
group by 2, w.productid) as data) as rating
where
r <=5
```

---

Write a query to find year to date revenue for each city of delivery and year.

```
select distinct extract(year from soh.shipdate) as year,
a.city,
soh.shipdate,
sum(soh.subtotal) over(partition by extract(year from soh.shipdate),
a.city order by soh.shipdate)
from sales.salesorderheader soh
join person.address a on a.addressid=soh.shiptoaddressid
order by 1, 2, 3;
```

Other options to change window frame

Over (partition by … order by ... ROWS BETWEEN ... AND ...)

set start & end of window frame inside a partition:

n preceding – start from previous n records (or less if not possible)  
n following – continue to n next records (or less if not possible)  
unbounded preceding – start from the partition’s beginning  
unbounded following – stop at the partition’s last record  
current row – use a row being currently processed at runtime  


__Accessing previous and next records__


Rewrite the following query to calculate revenue from current and 10 previous orders by

```
select
city,
date_part('y', shipdate) as ship_year,
shipdate,
sum(subtotal) over yr_city as total_YTD
from sales.salesorderheader as sh
join person.address as a
on addressid=shiptoaddressid
window yr_city
as ( partition by city,date_part('y', shipdate) order by shipdate)
order by
city,
date_part('y', shipdate),
shipdate;
```

Corrected query:

```
select
city,
date_part('y', shipdate) as ship_year,
shipdate,
sum(subtotal) over yr_city as total_YTD
from sales.salesorderheader as sh
join person.address as a
on addressid=shiptoaddressid
window yr_city
as ( partition by city,date_part('y', shipdate) order by shipdate rows between 10
preceding and current row)
order by
city,
date_part('y', shipdate),
shipdate;
```

Find month-to-month difference in number of orders month of due date, 
Use LAG(expression, relative_position) OVER() or LEAD(expression, relative_position)
OVER() function. You also can add the third argument to set a value which will be returned if the
result cannot be calculated (e.g., when position argument points outside current partition).

```
select pc.name,
date_part('y', sh.duedate) as yr,
date_part('mon', sh.duedate) as mon,
count(distinct sh.salesorderid) as number_of_orders
from sales.salesorderheader as sh
join sales.salesorderdetail as sd using (salesorderid)
join production.product p using(productid)
join production.productsubcategory ps using (productsubcategoryid)
join production.productcategory pc using (productcategoryid)
group by 1, 2, 3
```


```
select
yr,
mon,
name as category,
number_of_orders - lag(number_of_orders, 1) over(partition by name order by yr,
mon) as difference
from
(select pc.name,
date_part('y', sh.duedate) as yr,
date_part('mon', sh.duedate) as mon,
count(distinct sh.salesorderid) as number_of_orders
from sales.salesorderheader as sh
join sales.salesorderdetail as sd using (salesorderid)
join production.product p using(productid)
join production.productsubcategory ps using (productsubcategoryid)
join production.productcategory pc using (productcategoryid)
group by 1, 2, 3) as data
order by 3, 1, 2
```

__CTE Common Table Expression__

Prepare a query for a report with the following structure:
• Country code (billing address)
• Product category
• Number of orders
• Taxes payed
Show only orders made in 2012. If there were no orders for a particular combination of
category and year then display 0 instead.
Find all countries that had 100 or less orders in clothes category in 2012.

```
with c_c as (
select * from
(select distinct countryregioncode country from address) c
cross join
(select productcategoryid id, name cname from productcategory) p),
orders as (
select
sum(sh.taxamt) as taxes,
count(distinct sh.salesorderid) as ordersnr,
a.countryregioncode as country,
pc."name" as cname,
pc.productcategoryid as id,
sum(count(distinct case pc."name" when 'Clothing' then sh.salesorderid else null
end))
over (partition by a.countryregioncode) as cl_ord_nr
from salesorderheader as sh
join salesorderdetail as sd using (salesorderid)
join product p using(productid)
join productsubcategory ps using (productsubcategoryid)
join productcategory pc using (productcategoryid)
join address a on a.addressid = sh.billtoaddressid
where extract(year from sh.duedate ) = 2012
group by 3, 4, 5
)
select c_c.country,
c_c.cname,
coalesce(ordersnr, 0) ordersnr,
coalesce(taxes, 0) taxes
from
c_c left join orders
on c_c.country = orders.country
and c_c.id = orders.id
where cl_ord_nr <= 100 or cl_ord_nr is null
```

- auto upload, disable editor area, show loading progress  
- 尽量显示全文件名，上传的时候也是，在文件名很长的时候  
- 编辑附件应该是 replace 或者 delete
- 优化 thread preview images, hide small， 圆角

- user activities
"hanson liked winter's comment in <topic_topic>

hanson liked winter's topic <topic_title>

hanson replied winter's comment in <topic_title>

hanson replied to winter's topic <topic_title>

- sider footer 调整，和分类见巨大一点，文字有点大，参考上面  member，online
- 排序分类右边点点在 hover下面隐藏起来了，
- 最好hover的情况是整个block
- 左边新帖子的点点在鼠标移上去才出来
- 左边的点点需要大一点，zheqi设计